# Approach 1 - Simple conditional modelling for strict rules 

In [20]:
import pandas as pd
import numpy as np

# --- Load ---
df = pd.read_csv("C:/Users/MOHAMMED ARBAZ/Downloads/Amazon Sale Report.csv/Amazon Sale Report.csv")

# --- Normalize column names ---
def normalize_col(c):
    return (
        str(c).strip()
        .replace("\xa0", " ")
        .replace("\u200b", "")
        .strip()
        .lower()
        .replace(" ", "_")
        .replace("-", "_")
    )

df.columns = [normalize_col(c) for c in df.columns]

# --- Flag promo rows (where promotion_ids is NOT null/blank) ---
df["promo_flag"] = df["promotion_ids"].notna().astype(int) if "promotion_ids" in df.columns else 0

# --- Ensure numeric fields ---
df["qty"] = pd.to_numeric(df["qty"], errors="coerce")
df["amount"] = pd.to_numeric(df["amount"], errors="coerce")

# --- Clean Status and Remove Cancelled ---
if "status" in df.columns:
    status_clean = df["status"].astype(str).str.strip().str.lower()
    df = df[~status_clean.isin(["cancelled", "canceled"])].copy()

# Drop rows with invalid qty or amount
df = df.dropna(subset=["amount"])
df = df[df["qty"] > 0].copy()

# --- Compute MRP ---
df["mrp"] = df["amount"] / df["qty"]
df = df[np.isfinite(df["mrp"]) & (df["mrp"] > 0)].copy()

# --- Drop null SKU or Category ---
df = df.dropna(subset=["sku", "category"]).copy()
df["sku"] = df["sku"].astype(str).str.strip()
df["category"] = df["category"].astype(str).str.strip()

# --- Add B2B flag columns ---
df["b2b_qty"] = np.where(df["b2b"].isna(), df["qty"], 0)
df["non_b2b_qty"] = np.where(df["b2b"].astype(str).str.lower() == "false", df["qty"], 0)

# --- Group by SKU, Category, MRP, Date ---
grouped = (
    df.groupby(["sku", "category", "mrp", "date"], as_index=False)
      .agg(
          qty_sold=("qty", "sum"),
          revenue=("amount", "sum"),
          orders=("order_id", "nunique") if "order_id" in df.columns else ("mrp", "count"),
          promo_qty=("qty", lambda x: x[df.loc[x.index, "promo_flag"] == 1].sum()),
          b2b_qty=("b2b_qty", "sum"),
          non_b2b_qty=("non_b2b_qty", "sum")
      )
)

# ============================
# === Status breakdown
# ============================

# Canonical status map
status_map = {
    "cancelled": "cancelled",
    "canceled": "cancelled",

    "shipped - delivered to buyer": "delivered_to_buyer",
    "shipped": "intransit",
    "shipped - returned to seller": "returned_to_seller",
    "shipped - rejected by buyer": "rejected_by_buyer",
    "shipped - picked up": "intransit",
    "shipped - out for delivery": "intransit",
    "shipped - lost in transit": "lost_in_transit",
    "shipped - returning to seller": "returning_to_seller",
    "shipped - damaged": "damaged",
    "pending": "pending",
    "pending - waiting for pick up": "waiting_for_pickup",
    "shipping": "intransit",
}

status_clean_raw = df["status"].astype(str).str.strip().str.lower()
df["status_clean"] = status_clean_raw.map(status_map).fillna("other")

expected_status_cols = [
    "cancelled",
    "delivered_to_buyer",
    "intransit",
    "returned_to_seller",
    "rejected_by_buyer",
    "lost_in_transit",
    "returning_to_seller",
    "pending",
    "waiting_for_pickup",
    "damaged",
    "shipping",
    "other"
]

# Group and Pivot qty by status
status_qty = (
    df.groupby(["sku", "category", "mrp", "date", "status_clean"], as_index=False)["qty"]
      .sum()
      .rename(columns={"qty": "qty_sum"})
)

pivot_status = (
    status_qty
    .pivot(index=["sku", "category", "mrp", "date"], columns="status_clean", values="qty_sum")
    .fillna(0)
)

# Ensure all expected columns exist
for col in expected_status_cols:
    if col not in pivot_status.columns:
        pivot_status[col] = 0

# Reorder + cast to int
pivot_status = pivot_status[expected_status_cols].astype(int).reset_index()

# ============================
# === Final Merge
# ============================

final_product_data = grouped.merge(
    pivot_status,
    on=["sku", "category", "mrp", "date"],
    how="outer"
)

# final_product_data is ready
print(final_product_data.head())
print(final_product_data.shape)


import pandas as pd
import numpy as np

# Assuming `final_product_data` is already available
df = final_product_data.copy()

# --- Step 1: Compute shares ---
df["promo_share"] = df["promo_qty"] / df["qty_sold"]
df["b2b_share"] = df["b2b_qty"] / df["qty_sold"]
df["non_b2b_share"] = df["non_b2b_qty"] / df["qty_sold"]

# --- Step 2: Assume COGS = 40% of MRP ---
df["cogs"] = df["mrp"] * 0.40

# --- Step 3: Define elasticity logic ---
def suggest_price_adjustment(row):
    promo = row["promo_share"]
    b2b = row["b2b_share"]
    cogs = row["cogs"]
    mrp = row["mrp"]

    # Case 1: Heavy promo + B2B reliance
    if promo > 0.6 and b2b > 0.6:
        return mrp * 0.85  # 15% reduction

    # Case 2: Promo heavy but B2B low
    elif promo > 0.6 and b2b <= 0.3:
        return mrp * 0.90

    # Case 3: B2B heavy but Promo low
    elif b2b > 0.6 and promo <= 0.3:
        return mrp * 0.90

    # Case 4: Healthy mix
    elif promo < 0.3 and b2b < 0.3:
        return mrp * 1.05  # Suggest 5% increase

    # Default: No change
    return mrp

df["suggested_mrp"] = df.apply(suggest_price_adjustment, axis=1)

# --- Output example ---
output = df[["sku", "category", "date", "mrp", "qty_sold", "promo_share", "b2b_share", "suggested_mrp"]]
print(output.head())

df.to_csv('conditional_mrp_for_sku.csv',index=False)


             sku category    mrp      date  qty_sold  revenue  orders  \
0    AN201-RED-M   Bottom  229.0  04-07-22         1    229.0       1   
1    AN201-RED-M   Bottom  229.0  05-01-22         1    229.0       1   
2   AN201-RED-XL   Bottom  301.0  06-24-22         1    301.0       1   
3   AN201-RED-XL   Bottom  301.0  06-26-22         1    301.0       1   
4  AN201-RED-XXL   Bottom  229.0  05-03-22         1    229.0       1   

   promo_qty  b2b_qty  non_b2b_qty  ...  intransit  returned_to_seller  \
0          1        0            1  ...          1                   0   
1          1        0            1  ...          0                   0   
2          1        0            1  ...          1                   0   
3          0        0            1  ...          1                   0   
4          1        0            1  ...          0                   0   

   rejected_by_buyer  lost_in_transit  returning_to_seller  pending  \
0                  0                0        

In [25]:
# ============================
# === PREP FOR MACHINE LEARNING (FIXED) ===
# ============================

# Always use the aggregated table (has qty_sold, promo_qty, status cols, etc.)
df_ml = final_product_data.copy()

# --- Ensure required columns exist; if missing, create zeros ---
for c in ["promo_qty", "b2b_qty", "non_b2b_qty", "qty_sold"]:
    if c not in df_ml.columns:
        df_ml[c] = 0

# Some datasets may miss category; if so, synthesize a default
if "category" not in df_ml.columns:
    df_ml["category"] = "ALL"

# --- Encode categorical variables (handle NaN safely) ---
from sklearn.preprocessing import LabelEncoder

df_ml["sku"] = df_ml["sku"].astype(str).fillna("UNK")
df_ml["category"] = df_ml["category"].astype(str).fillna("UNK")

le_sku = LabelEncoder()
le_cat = LabelEncoder()
df_ml["sku_enc"] = le_sku.fit_transform(df_ml["sku"])
df_ml["category_enc"] = le_cat.fit_transform(df_ml["category"])

# --- Date features (robust parse then month/year) ---
df_ml["date"] = pd.to_datetime(df_ml["date"], errors="coerce")
df_ml["month"] = df_ml["date"].dt.month.fillna(0).astype(int)
df_ml["year"]  = df_ml["date"].dt.year.fillna(0).astype(int)

# --- Safe division helper to prevent /0 and NaNs ---
def safe_div(numer, denom):
    return np.where(denom > 0, numer / denom, 0.0)

# --- Share features ---
df_ml["promo_share"]     = safe_div(df_ml["promo_qty"], df_ml["qty_sold"])
df_ml["b2b_share"]       = safe_div(df_ml["b2b_qty"], df_ml["qty_sold"])
df_ml["non_b2b_share"]   = safe_div(df_ml["non_b2b_qty"], df_ml["qty_sold"])

# --- Status percentage features (create if missing, then pct) ---
status_cols = [
    "cancelled", "delivered_to_buyer", "intransit", "returned_to_seller",
    "rejected_by_buyer", "lost_in_transit", "returning_to_seller",
    "pending", "waiting_for_pickup", "damaged", "shipping", "other"
]
for col in status_cols:
    if col not in df_ml.columns:
        df_ml[col] = 0
    df_ml[f"{col}_pct"] = safe_div(df_ml[col], df_ml["qty_sold"])

# --- Final Features and Target ---
base_features = ["sku_enc", "category_enc", "month", "year",
                 "promo_share", "b2b_share", "non_b2b_share"]
status_pct_feats = [f"{col}_pct" for col in status_cols]

# Keep only columns that truly exist (defensive)
features = [f for f in base_features + status_pct_feats if f in df_ml.columns]

# Target is the price level (mrp) present in the aggregated table
if "mrp" not in df_ml.columns:
    raise ValueError("Target 'mrp' column not found in final_product_data. Check earlier grouping step.")
target = "mrp"

X = df_ml[features].fillna(0)
y = pd.to_numeric(df_ml[target], errors="coerce")
mask = y.notna()
X, y = X[mask], y[mask]

# --- Train/Test Split ---
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# --- Try Multiple Regressors ---
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

models = {
    "Linear": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=42)
}

rows = []
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    r2  = r2_score(y_test, preds) if np.isfinite(y_test).all() else np.nan
    rows.append((name, mse, r2))

results_df = pd.DataFrame(rows, columns=["Model", "MSE", "R2"]).sort_values("MSE")
print("\nModel performance on held-out set:")
print(results_df.to_string(index=False))



Model performance on held-out set:
       Model          MSE       R2
RandomForest  1904.544120 0.974676
       Lasso 57832.535643 0.231012
       Ridge 57841.478870 0.230893
      Linear 57841.517316 0.230892


### checking the count of sku which have the sales consistently over 6 months

In [26]:
import pandas as pd

# Load
df = pd.read_csv(r"C:/Users/MOHAMMED ARBAZ/Downloads/Amazon Sale Report.csv/Amazon Sale Report.csv")

# Parse dates robustly (handles 05-02-22 and 01-05-2022 etc.)
df['date'] = pd.to_datetime(df['Date'].astype(str).str.strip(), dayfirst=True, errors='coerce')

# Drop rows where date couldn't be parsed
df = df.dropna(subset=['date'])

# Min/Max per SKU
rng = df.groupby('SKU')['date'].agg(min_date='min', max_date='max')

# Month span (year*12 + month) and adjust if the end day is earlier than the start day
month_span = (
    (rng['max_date'].dt.year - rng['min_date'].dt.year) * 12
    + (rng['max_date'].dt.month - rng['min_date'].dt.month)
    - (rng['max_date'].dt.day < rng['min_date'].dt.day).astype(int)
)

# SKUs with span > 6 months
result = rng.assign(months_span=month_span)
result_over6 = result[result['months_span'] > 6]

count_sku = result_over6.shape[0]

print("SKUs with span > 6 months:")
print(result_over6)
print("Count of SKUs:", count_sku)


SKUs with span > 6 months:
                    min_date   max_date  months_span
SKU                                                 
BL006-54BLACK     2022-03-04 2022-12-05            9
BL009-61BLACK     2022-01-04 2022-09-05            8
BL009-61BLACK-B   2022-01-05 2022-10-06            9
BL013-62BLACK     2022-03-05 2022-12-05            9
BL035-161GOLD     2022-01-04 2022-12-05           11
...                      ...        ...          ...
SET433-KR-NP-XXXL 2022-03-06 2022-11-04            7
SET435-KR-NP-L    2022-01-06 2022-09-06            8
SET436-KR-NP-M    2022-01-06 2022-10-05            8
SET436-KR-NP-XS   2022-04-06 2022-12-06            8
SET436-KR-NP-XXXL 2022-01-05 2022-12-06           11

[3224 rows x 3 columns]
Count of SKUs: 3224


# custom regressor model 


In [27]:
# ===========================================
# PRICE OPTIMIZATION + SARIMAX FORECAST (PER SKU) with DEBUG + HOLDOUT
# ===========================================

import warnings, time, os
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# SARIMAX
try:
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    HAS_SARIMAX = True
except Exception:
    HAS_SARIMAX = False

# Spline (optional)
try:
    import statsmodels.api as sm
    from patsy import dmatrix
    HAS_PATSY = True
except Exception:
    HAS_PATSY = False

# -------------------------------
# CONFIG
# -------------------------------
CSV_PATH = r"C:/Users/MOHAMMED ARBAZ/Downloads/Amazon Sale Report.csv/Amazon Sale Report.csv"

# Costs
COST_COLUMN = None
COST_FRACTION_OF_PRICE = 0.70

# Price grid / compute
PRICE_BOUND_PCT = 0.25
N_PRICE_GRID = 25
FORECAST_HORIZON = 30

# SARIMAX params
SARIMAX_ORDER = (1, 1, 1)
SARIMAX_SEASONAL_ORDER = (1, 1, 1, 7)
SARIMAX_VAL_DAYS_MIN = 14   # min days for SARIMAX validation

# Data thresholds
MIN_DAILY_ROWS_SKU = 14
MIN_DAILY_ROWS_SARIMA = 30

# Holdout split for regressors
HOLDOUT_FRACTION = 0.20     # last 20% of days as test; min 7 days enforced

# Debug / verbosity
VERBOSE = True
PRINT_EVERY_SKU = 10        # print progress every N SKUs

def log(msg):
    if VERBOSE:
        print(msg, flush=True)

t0 = time.time()
log("=== Step 0: Imports OK. Starting pipeline ===")

# -------------------------------
# 1) LOAD & CLEAN
# -------------------------------
def normalize_col(c):
    return (
        str(c).strip()
        .replace("\xa0", " ")
        .replace("\u200b", "")
        .strip()
        .lower()
        .replace(" ", "_")
        .replace("-", "_")
    )

log(f"Reading CSV: {CSV_PATH}")
df_raw = pd.read_csv(CSV_PATH)
df_raw.columns = [normalize_col(c) for c in df_raw.columns]
df_raw = df_raw.drop(columns=['index'])
log(f"Raw shape: {df_raw.shape}")
log(f"Columns: {list(df_raw.columns)}")

date_col = "date" if "date" in df_raw.columns else ("order_date" if "order_date" in df_raw.columns else None)
if date_col is None:
    raise ValueError("No date column found ('date' or 'order_date').")

sku_col = "sku"
qty_col = "qty" if "qty" in df_raw.columns else ("quantity" if "quantity" in df_raw.columns else None)
amount_col = "amount" if "amount" in df_raw.columns else ("revenue" if "revenue" in df_raw.columns else None)
cat_col = "category" if "category" in df_raw.columns else None
if not all([sku_col in df_raw.columns, qty_col, amount_col]):
    raise ValueError("Missing required columns: sku, qty/quantity, amount/revenue.")

# Parse dates (month-first pref, fallback to day-first)
d_pref = pd.to_datetime(df_raw[date_col].astype(str).str.strip(), errors="coerce", dayfirst=False)
d_fall = pd.to_datetime(df_raw[date_col].astype(str).str.strip(), errors="coerce", dayfirst=True)
df = df_raw.copy()
df["date_parsed"] = d_pref.fillna(d_fall)
df["day"] = pd.to_datetime(df["date_parsed"].dt.date)
n_bad_dates = df["day"].isna().sum()
log(f"Parsed dates. Bad/missing dates: {n_bad_dates}")

# numerics & price
df[qty_col] = pd.to_numeric(df[qty_col], errors="coerce")
df[amount_col] = pd.to_numeric(df[amount_col], errors="coerce")
before = df.shape[0]
df = df.dropna(subset=["day", qty_col, amount_col])
df = df[(df[qty_col] > 0) & (df[amount_col] > 0)].copy()
df["price"] = df[amount_col] / df[qty_col]
df = df[np.isfinite(df["price"]) & (df["price"] > 0)].copy()
log(f"Cleaned numerics & price. Dropped {before - df.shape[0]} rows. Remaining: {df.shape[0]}")

# promo flag
promo_col = "promotion_ids" if "promotion_ids" in df.columns else None
df["promo_flag"] = df[promo_col].notna().astype(int) if promo_col else 0

# drop canceled if present
if "status" in df.columns:
    s = df["status"].astype(str).str.lower().str.strip()
    before = df.shape[0]
    df = df[~s.isin(["cancelled", "canceled"])]
    log(f"Dropped cancelled rows: {before - df.shape[0]}")

# categories fallback
if cat_col is None:
    df["category"] = "ALL"
    cat_col = "category"

df[sku_col] = df[sku_col].astype(str).str.strip()
df[cat_col] = df[cat_col].astype(str).str.strip()

# -------------------------------
# 2) DAILY AGG PER SKU
# -------------------------------
daily = (
    df.groupby([sku_col, cat_col, "day"], as_index=False)
      .agg(qty=(qty_col, "sum"),
           revenue=(amount_col, "sum"),
           price=("price", "median"),
           promo_flag=("promo_flag", "max"))
)
daily["dow"] = daily["day"].dt.dayofweek
daily["month"] = daily["day"].dt.month
daily["ln_q"] = np.log(daily["qty"] + 1e-6)
daily["ln_p"] = np.log(daily["price"])

dow_d = pd.get_dummies(daily["dow"], prefix="dow", drop_first=True)
mon_d = pd.get_dummies(daily["month"], prefix="m", drop_first=True)
daily = pd.concat([daily, dow_d, mon_d], axis=1)
CTRL_COLS = [c for c in daily.columns if c.startswith("dow_") or c.startswith("m_")] + ["promo_flag"]

n_skus = daily[[sku_col, cat_col]].drop_duplicates().shape[0]
log(f"Daily panel ready. Rows: {daily.shape[0]}, Unique SKUs: {n_skus}")

# SKU price stats
sku_price_stats = (
    daily.sort_values("day")
         .groupby([sku_col, cat_col])
         .agg(latest_price=("price", "last"),
              min_price=("price", "min"),
              max_price=("price", "max"),
              n_days=("day", "nunique"))
         .reset_index()
)

# costs (prefer actual column)
if COST_COLUMN and COST_COLUMN in df.columns:
    last_cost = (
        df.sort_values("day")
          .groupby([sku_col, cat_col])
          .tail(1)[[sku_col, cat_col, COST_COLUMN]]
          .rename(columns={COST_COLUMN: "unit_cost"})
    )
    sku_price_stats = sku_price_stats.merge(last_cost, on=[sku_col, cat_col], how="left")
else:
    sku_price_stats["unit_cost"] = np.nan  # fill later from fraction

def make_price_grid(latest, lo_obs, hi_obs, pct=PRICE_BOUND_PCT, n=N_PRICE_GRID):
    lo = max(0.01, latest * (1 - pct), lo_obs)
    hi = max(lo + 1e-6, latest * (1 + pct), hi_obs)
    return np.linspace(lo, hi, n)

# -------------------------------
# 3) PRICE->QTY MODELS + HOLDOUT
# -------------------------------
def time_split(g):
    """Return g_train, g_test (time-based split)."""
    g = g.sort_values("day")
    unique_days = g["day"].drop_duplicates().tolist()
    n = len(unique_days)
    n_test = max(7, int(np.ceil(n * HOLDOUT_FRACTION)))
    if n <= n_test + 5:  # ensure train not too tiny
        return g, None
    cutoff = unique_days[-n_test]
    g_train = g[g["day"] < cutoff].copy()
    g_test = g[g["day"] >= cutoff].copy()
    return g_train, g_test

def fit_loglog(g):
    X = g[["ln_p"] + CTRL_COLS]
    y = g["ln_q"]
    if len(g) < 8:
        return None
    lr = LinearRegression().fit(X, y)
    return ("loglog", lr, X.columns.tolist())

def fit_loglog_quad(g):
    X = g[["ln_p"]].copy()
    X["ln_p2"] = X["ln_p"]**2
    X = pd.concat([X, g[CTRL_COLS]], axis=1)
    y = g["ln_q"]
    if len(g) < 10:
        return None
    lr = LinearRegression().fit(X, y)
    return ("loglog_quad", lr, X.columns.tolist())

def fit_spline(g):
    if not HAS_PATSY:
        return None
    design = dmatrix("bs(ln_p, df=4, include_intercept=True)", data=g, return_type="dataframe")
    X = pd.concat([design, g[CTRL_COLS]], axis=1)
    y = g["ln_q"]
    if len(g) < 12:
        return None
    model = sm.OLS(y, sm.add_constant(X, has_constant="add")).fit()
    return ("spline", model, X.columns.tolist(), True)

def fit_rf(g):
    X = g[["price"] + CTRL_COLS]
    y = g["qty"]
    if len(g) < 12:
        return None
    rf = RandomForestRegressor(n_estimators=300, random_state=42)
    rf.fit(X, y)
    return ("rf", rf, X.columns.tolist())

def predict_qty_from_model(model_tuple, dfX):
    name = model_tuple[0]
    if name in ("loglog", "loglog_quad"):
        lr, cols = model_tuple[1], model_tuple[2]
        dfX = dfX.reindex(columns=cols, fill_value=0.0)
        ln_q = lr.predict(dfX.values)
        return np.exp(ln_q).clip(min=0.0)
    if name == "spline":
        model, cols, _ = model_tuple[1], model_tuple[2], model_tuple[3]
        dfX = sm.add_constant(dfX.reindex(columns=cols, fill_value=0.0), has_constant="add")
        ln_q = model.predict(dfX)
        return np.exp(ln_q).clip(min=0.0)
    if name == "rf":
        rf, cols = model_tuple[1], model_tuple[2]
        dfX = dfX.reindex(columns=cols, fill_value=0.0)
        q = rf.predict(dfX.values)
        return np.maximum(q, 0.0)
    raise ValueError("Unknown model")

def build_grid_frame(price_grid, avg_ctrls, model_name):
    if model_name in ("loglog", "loglog_quad", "spline"):
        dfX = pd.DataFrame({"ln_p": np.log(price_grid)})
        if model_name == "loglog_quad":
            dfX["ln_p2"] = dfX["ln_p"]**2
        for c in CTRL_COLS:
            dfX[c] = avg_ctrls.get(c, 0.0)
        return dfX
    else:
        dfX = pd.DataFrame({"price": price_grid})
        for c in CTRL_COLS:
            dfX[c] = avg_ctrls.get(c, 0.0)
        return dfX

def eval_holdout(model_tuple, g_train, g_test):
    """Return dict of holdout metrics for model_tuple."""
    if g_test is None or len(g_test) == 0:
        return {}
    name = model_tuple[0]
    if name in ("loglog", "loglog_quad", "spline"):
        if name == "loglog":
            Xtest = g_test[["ln_p"] + CTRL_COLS]
        elif name == "loglog_quad":
            Xtest = g_test[["ln_p"]].copy()
            Xtest["ln_p2"] = Xtest["ln_p"]**2
            Xtest = pd.concat([Xtest, g_test[CTRL_COLS]], axis=1)
        else:  # spline
            design = dmatrix("bs(ln_p, df=4, include_intercept=True)", data=g_test, return_type="dataframe")
            Xtest = pd.concat([design, g_test[CTRL_COLS]], axis=1)
        y_true_ln = g_test["ln_q"].values
        q_pred = predict_qty_from_model(model_tuple, Xtest)
        y_pred_ln = np.log(q_pred + 1e-6)
        r2_log = r2_score(y_true_ln, y_pred_ln) if np.isfinite(y_true_ln).all() else np.nan
        mae_qty = mean_absolute_error(g_test["qty"].values, q_pred)
        return {"r2_log": round(float(r2_log), 4) if np.isfinite(r2_log) else np.nan,
                "mae_qty": round(float(mae_qty), 4)}
    else:
        Xtest = g_test[["price"] + CTRL_COLS]
        q_true = g_test["qty"].values
        q_pred = predict_qty_from_model(model_tuple, Xtest)
        r2_qty = r2_score(q_true, q_pred) if len(np.unique(q_true)) > 1 else np.nan
        mae_qty = mean_absolute_error(q_true, q_pred)
        return {"r2_qty": round(float(r2_qty), 4) if np.isfinite(r2_qty) else np.nan,
                "mae_qty": round(float(mae_qty), 4)}

# -------------------------------
# 4) SARIMAX helpers (with validation)
# -------------------------------
def sarimax_fit_and_validate(g):
    """Return (res, val_metrics_dict). Fit on train, validate on last SARIMAX_VAL_DAYS_MIN days."""
    if not HAS_SARIMAX or len(g) < max(MIN_DAILY_ROWS_SARIMA, SARIMAX_VAL_DAYS_MIN+10):
        return None, {}
    g = g.sort_values("day")
    unique_days = g["day"].drop_duplicates().tolist()
    if len(unique_days) < SARIMAX_VAL_DAYS_MIN + 10:
        return None, {}

    cutoff = unique_days[-SARIMAX_VAL_DAYS_MIN]
    g_train = g[g["day"] < cutoff]
    g_test = g[g["day"] >= cutoff]
    if len(g_train) < MIN_DAILY_ROWS_SARIMA:
        return None, {}

    s_tr = g_train.set_index("day").asfreq("D")
    for c in ["qty", "price", "promo_flag"]:
        if c not in s_tr.columns: s_tr[c] = 0
    s_tr["qty"] = s_tr["qty"].fillna(0)
    s_tr["price"] = s_tr["price"].ffill()
    s_tr["promo_flag"] = s_tr["promo_flag"].fillna(0)
    y_tr = s_tr["qty"]
    ex_tr = s_tr[["price", "promo_flag"]]

    try:
        mod = SARIMAX(y_tr, order=SARIMAX_ORDER, seasonal_order=SARIMAX_SEASONAL_ORDER,
                      exog=ex_tr, enforce_stationarity=False, enforce_invertibility=False)
        res = mod.fit(disp=False)
    except Exception:
        return None, {}

    # validate on test using actual future exog
    s_te = g_test.set_index("day").asfreq("D")
    s_te["qty"] = s_te["qty"].fillna(0)
    s_te["price"] = s_te["price"].ffill()
    s_te["promo_flag"] = s_te["promo_flag"].fillna(0)
    ex_te = s_te[["price", "promo_flag"]]
    try:
        fc = res.get_forecast(steps=len(s_te), exog=ex_te)
        pred = np.maximum(fc.predicted_mean.values, 0.0)
        true = s_te["qty"].values
        rmse = mean_squared_error(true, pred, squared=False)
        mape = np.mean(np.abs((true - pred) / np.maximum(true, 1e-6))) * 100.0
        return res, {"sarimax_rmse": round(float(rmse), 4), "sarimax_mape": round(float(mape), 2)}
    except Exception:
        return res, {}

def sarimax_forecast_for_price_with_model(res, g, future_price, horizon=FORECAST_HORIZON):
    """Use a fitted SARIMAX result (res) to forecast next horizon with constant future price."""
    s = g.set_index("day").asfreq("D")
    s["qty"] = s["qty"].fillna(0)
    s["price"] = s["price"].ffill()
    s["promo_flag"] = s["promo_flag"].fillna(0)
    y = s["qty"]
    # refresh state with full history
    try:
        res_full = res.model.smooth(res.params)
    except Exception:
        res_full = res
    future_idx = pd.date_range(y.index[-1] + pd.Timedelta(days=1), periods=horizon, freq="D")
    future_exog = pd.DataFrame({"price": [future_price]*horizon, "promo_flag": [0]*horizon}, index=future_idx)
    try:
        fc = res_full.get_forecast(steps=horizon, exog=future_exog)
        return np.maximum(fc.predicted_mean.values, 0.0)
    except Exception:
        return None

# -------------------------------
# 5) PER-SKU LOOP
# -------------------------------
records = []
skus = daily[[sku_col, cat_col]].drop_duplicates().values.tolist()
log(f"Starting per-SKU loop over {len(skus)} SKUs...")

for idx, (sku, cat) in enumerate(skus, 1):
    g = daily[(daily[sku_col]==sku) & (daily[cat_col]==cat)].sort_values("day").copy()
    stats_row = sku_price_stats[(sku_price_stats[sku_col]==sku) & (sku_price_stats[cat_col]==cat)]
    if stats_row.empty:
        continue

    latest_p = float(stats_row["latest_price"])
    min_p = float(stats_row["min_price"])
    max_p = float(stats_row["max_price"])
    n_days = int(stats_row["n_days"])
    unit_cost = stats_row["unit_cost"].values[0] if "unit_cost" in stats_row.columns else np.nan
    if not np.isfinite(unit_cost):
        unit_cost = latest_p * COST_FRACTION_OF_PRICE

    if idx % PRINT_EVERY_SKU == 0 or idx == 1:
        log(f"[{idx}/{len(skus)}] SKU={sku} | Cat={cat} | n_days={n_days} | latest={latest_p:.2f}")

    # SARIMAX fit + validation (once per SKU)
    sarimax_res, sarimax_val = sarimax_fit_and_validate(g)
    if sarimax_res is None:
        log(f"  SARIMAX: not enough data or fit failed for {sku}")

    # Skip if too little data for price models
    if n_days < MIN_DAILY_ROWS_SKU:
        if sarimax_res is not None:
            fc = sarimax_forecast_for_price_with_model(sarimax_res, g, latest_p)
            if fc is not None:
                records.append({
                    "sku": sku, "category": cat, "model": "SARIMAX_baseline",
                    "optimal_price": round(latest_p,2),
                    "expected_30d_qty": round(float(fc.sum()),2),
                    "expected_30d_profit": round(float((latest_p - unit_cost) * fc.sum()),2),
                    "latest_price": latest_p,
                    **sarimax_val,
                    "note": "Baseline at current price"
                })
        records.append({
            "sku": sku, "category": cat, "model": "insufficient_data",
            "optimal_price": np.nan, "expected_30d_qty": np.nan, "expected_30d_profit": np.nan,
            "latest_price": latest_p, **sarimax_val, "note": f"Only {n_days} daily rows"
        })
        continue

    # Holdout split for price->qty models
    def time_split_local(g_local):
        gtr, gte = time_split(g_local)
        return gtr, gte

    g_train, g_test = time_split_local(g)
    avg_ctrls_train = {c: float(g_train[c].mean()) for c in CTRL_COLS}

    # Fit price->qty models on train
    fitted = []
    for fitter in (fit_loglog, fit_loglog_quad, fit_spline, fit_rf):
        try:
            res_m = fitter(g_train)
            if res_m is not None:
                fitted.append(res_m)
                log(f"  Fitted {res_m[0]} on train (rows={len(g_train)})")
        except Exception as e:
            log(f"  Fit error {fitter.__name__}: {e}")

    # Validate models on test
    model_metrics = {}
    for m in fitted:
        try:
            metrics = eval_holdout(m, g_train, g_test)
            model_metrics[m[0]] = metrics
            if metrics:
                log(f"  Holdout {m[0]}: {metrics}")
        except Exception as e:
            log(f"  Holdout error {m[0]}: {e}")

    # Candidate price grid
    price_grid = make_price_grid(latest_p, min_p, max_p, pct=PRICE_BOUND_PCT, n=N_PRICE_GRID)

    # For each model: evaluate candidate prices via SARIMAX (if fitted)
    for m in fitted:
        best_price = None
        best_profit = -np.inf
        best_qty_sum = 0.0

        if sarimax_res is None:
            # fallback quick eval: use price->qty model directly (avg daily * 30)
            try:
                grid_frame = build_grid_frame(price_grid, avg_ctrls_train, m[0])
                q_daily = predict_qty_from_model(m, grid_frame)
                q_30d = np.maximum(q_daily * FORECAST_HORIZON, 0.0)
                profit_30d = (price_grid - unit_cost) * q_30d
                j = int(np.argmax(profit_30d))
                best_price, best_profit, best_qty_sum = price_grid[j], profit_30d[j], q_30d[j]
            except Exception:
                pass
        else:
            # preferred: SARIMAX 30d forecast at each candidate price
            for p in price_grid:
                fc = sarimax_forecast_for_price_with_model(sarimax_res, g, p)
                if fc is None:
                    continue
                qty_30d = float(fc.sum())
                profit_30d = (p - unit_cost) * qty_30d
                if profit_30d > best_profit:
                    best_profit = profit_30d
                    best_price = p
                    best_qty_sum = qty_30d

        # ensure metric keys exist to avoid KeyError later
        row_metrics = {
            "r2_log": np.nan, "r2_qty": np.nan, "mae_qty": np.nan,
            "sarimax_rmse": np.nan, "sarimax_mape": np.nan
        }
        row_metrics.update(model_metrics.get(m[0], {}))
        row_metrics.update(sarimax_val if sarimax_val else {})

        # record
        records.append({
            "sku": sku, "category": cat, "model": m[0],
            "optimal_price": round(float(best_price), 2) if best_price is not None else np.nan,
            "expected_30d_qty": round(float(best_qty_sum), 2) if best_price is not None else np.nan,
            "expected_30d_profit": round(float(best_profit), 2) if best_price is not None else np.nan,
            "latest_price": latest_p,
            **row_metrics,
            "note": "SARIMAX-evaluated" if sarimax_res is not None else "Fallback: direct model"
        })

    # Also add a SARIMAX baseline (latest price) if we have SARIMAX
    if sarimax_res is not None:
        fc = sarimax_forecast_for_price_with_model(sarimax_res, g, latest_p)
        if fc is not None:
            base_metrics = {"sarimax_rmse": np.nan, "sarimax_mape": np.nan}
            base_metrics.update(sarimax_val if sarimax_val else {})
            records.append({
                "sku": sku, "category": cat, "model": "SARIMAX_baseline",
                "optimal_price": round(latest_p,2),
                "expected_30d_qty": round(float(fc.sum()),2),
                "expected_30d_profit": round(float((latest_p - unit_cost) * fc.sum()),2),
                "latest_price": latest_p,
                **base_metrics,
                "note": "Baseline at current price"
            })

log(f"Per-SKU loop done in {time.time()-t0:.1f}s")

# -------------------------------
# 6) OUTPUTS (ALL + BEST)
# -------------------------------
out = pd.DataFrame.from_records(records)
if out.empty:
    log("No results produced. Check data and thresholds.")
else:
    # ensure metric columns exist to avoid KeyErrors later
    for col in ["r2_log","r2_qty","mae_qty","sarimax_rmse","sarimax_mape"]:
        if col not in out.columns:
            out[col] = np.nan

    best_by_sku = (
        out.dropna(subset=["expected_30d_profit"])
           .sort_values(["sku","category","expected_30d_profit"], ascending=[True, True, False])
           .groupby(["sku","category"])
           .head(1)
    )

    out_all = "sku_price_optimization_all_models_with_sarimax_debug.csv"
    out_best = "sku_price_optimization_best_model_per_sku_debug.csv"
    out.to_csv(out_all, index=False)
    best_by_sku.to_csv(out_best, index=False)

    log("\n=== BEST MODEL PER SKU (top 10 by profit) ===")
    try:
        print(best_by_sku.sort_values("expected_30d_profit", ascending=False).head(10).to_string(index=False))
    except Exception:
        print(best_by_sku.head(10))

    log(f"\nSaved ALL model results: {os.path.abspath(out_all)}")
    log(f"Saved BEST per SKU:      {os.path.abspath(out_best)}")

# ===============================
# 7) ONE-LINE-PER-SKU SUMMARY
# ===============================
if out.empty:
    log("Skipping one-line summary because results are empty.")
else:
    # Base per-SKU aggregates from daily
    sku_agg = (
        daily.sort_values("day")
             .groupby([sku_col, cat_col])
             .agg(
                 total_qty=("qty","sum"),
                 total_revenue=("revenue","sum"),
                 min_price=("price","min"),
                 max_price=("price","max"),
                 avg_price=("price","mean"),
                 median_price=("price","median"),
                 n_days=("day","nunique"),
                 first_day=("day","min"),
                 last_day=("day","max"),
                 promo_days=("promo_flag","sum")
             )
             .reset_index()
    )
    sku_agg["promo_days_pct"] = (sku_agg["promo_days"] / sku_agg["n_days"]).fillna(0).round(4)

    # active months count
    months = daily.assign(ym=daily["day"].dt.to_period("M"))
    active_months = months.groupby([sku_col, cat_col])["ym"].nunique().reset_index().rename(columns={"ym":"active_months"})
    sku_agg = sku_agg.merge(active_months, on=[sku_col, cat_col], how="left")

    # best historical price by qty (price on the day with max qty)
    idx_max_qty_day = daily.groupby([sku_col, cat_col])["qty"].idxmax()
    best_hist = daily.loc[idx_max_qty_day, [sku_col, cat_col, "price", "qty"]].rename(
        columns={"price":"best_hist_price_by_qty", "qty":"best_hist_qty"}
    )

    # Merge with price stats (latest/min/max)
    sku_one_line = (sku_agg
        .merge(sku_price_stats[[sku_col, cat_col, "latest_price", "min_price", "max_price"]].drop_duplicates(),
               on=[sku_col, cat_col], how="left", suffixes=("","_dup"))
        .merge(best_hist, on=[sku_col, cat_col], how="left")
    )

    # Best model per SKU (ensure all metric columns exist)
    for col in ["r2_log","r2_qty","mae_qty","sarimax_rmse","sarimax_mape",
                "expected_30d_profit","expected_30d_qty","optimal_price"]:
        if col not in out.columns:
            out[col] = np.nan

    if not out.empty and "expected_30d_profit" in out.columns:
        best_model_cols = ["sku","category","model","optimal_price","expected_30d_qty","expected_30d_profit",
                           "r2_log","r2_qty","mae_qty","sarimax_rmse","sarimax_mape"]
        best_model_cols = [c for c in best_model_cols if c in out.columns]
        best_model = (out.dropna(subset=["expected_30d_profit"])
                        .sort_values(["sku","category","expected_30d_profit"], ascending=[True, True, False])
                        .groupby(["sku","category"])
                        .head(1)[best_model_cols]
                     )
        rename_map = {
            "model":"best_model_name",
            "optimal_price":"best_model_optimal_price",
            "expected_30d_qty":"best_model_expected_30d_qty",
            "expected_30d_profit":"best_model_expected_30d_profit"
        }
        best_model = best_model.rename(columns={k:v for k,v in rename_map.items() if k in best_model.columns})
        sku_one_line = sku_one_line.merge(best_model, on=[sku_col, cat_col], how="left")

    # Column order (only keep those present)
    cols_order = [
        sku_col, cat_col, "total_qty", "total_revenue",
        "min_price", "max_price", "avg_price", "median_price", "latest_price",
        "best_hist_price_by_qty", "best_hist_qty",
        "n_days", "active_months", "first_day", "last_day", "promo_days", "promo_days_pct",
        "best_model_name", "best_model_optimal_price",
        "best_model_expected_30d_qty", "best_model_expected_30d_profit",
        "r2_log", "r2_qty", "mae_qty", "sarimax_rmse", "sarimax_mape"
    ]
    cols_order = [c for c in cols_order if c in sku_one_line.columns]
    sku_one_line = sku_one_line[cols_order]

    # Save summary
    sku_summary_path = "sku_one_line_summary.csv"
    sku_one_line.to_csv(sku_summary_path, index=False)

    log(f"\nSaved ONE-LINE summary per SKU: {os.path.abspath(sku_summary_path)}")
    try:
        print(sku_one_line.head(10).to_string(index=False))
    except Exception:
        print(sku_one_line.head(10))


=== Step 0: Imports OK. Starting pipeline ===
Reading CSV: C:/Users/MOHAMMED ARBAZ/Downloads/Amazon Sale Report.csv/Amazon Sale Report.csv
Raw shape: (128975, 23)
Columns: ['order_id', 'date', 'status', 'fulfilment', 'sales_channel', 'ship_service_level', 'style', 'sku', 'category', 'size', 'asin', 'courier_status', 'qty', 'currency', 'amount', 'ship_city', 'ship_state', 'ship_postal_code', 'ship_country', 'promotion_ids', 'b2b', 'fulfilled_by', 'unnamed:_22']
Parsed dates. Bad/missing dates: 0
Cleaned numerics & price. Dropped 15274 rows. Remaining: 113701
Dropped cancelled rows: 5630
Daily panel ready. Rows: 74390, Unique SKUs: 7093
Starting per-SKU loop over 7093 SKUs...
[1/7093] SKU=AN201-RED-M | Cat=Bottom | n_days=2 | latest=229.00
  SARIMAX: not enough data or fit failed for AN201-RED-M
  SARIMAX: not enough data or fit failed for AN201-RED-XL
  SARIMAX: not enough data or fit failed for AN201-RED-XXL
  SARIMAX: not enough data or fit failed for AN202-ORANGE-M
  SARIMAX: not eno

KeyboardInterrupt: 